In [25]:
import re
import json

import numpy as np
from tqdm import tqdm

import faiss
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
with open("results/records_10.json", "r") as f:
    records_10 = json.load(f)

In [27]:
def cap_postprocess(cap):
    cap = cap.strip()
    cap = re.sub("^[\'\"]", "", cap)
    cap = re.sub("[\'\"]$", "", cap)
    return cap

records_embeds = {}
for key, captions in tqdm(records_10.items()):
    extracted_captions = []
    for caption in captions:
        caps = re.split("CAP\s*\d*\s*[:=-]", caption)[1:]
        caps = [cap_postprocess(cap) for cap in caps]
        extracted_captions.extend(caps)
    records_embeds[key] = [extracted_captions, model.encode(extracted_captions)]

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:25<00:00,  2.13s/it]


In [50]:
for key, (captions, embedding) in records_embeds.items():
    if key != "10.tif_32":
        continue
    break

In [78]:
kmeans = faiss.Kmeans(embedding.shape[1], 5, niter=20, verbose=True)
kmeans.train(embedding[:30])

10.926446914672852

In [79]:
index = faiss.IndexFlatL2(embedding.shape[1])
index.add(embedding)
D, I = index.search(kmeans.centroids, 2)

In [40]:
records_10.keys()

dict_keys(['10.tif_18', '10.tif_19', '10.tif_31', '10.tif_32', '10.tif_33', '10.tif_34', '10.tif_35', '10.tif_45', '10.tif_46', '10.tif_47', '10.tif_48', '10.tif_60'])

In [80]:
for i in range(I.shape[0]):
    print([captions[i] for i in I[i, :]])

['Top and bottom right buildings cluster.', 'Clustered buildings in top right and center.']
['Mostly horizontal/vertical alignment', 'Unique damage patterns top left and center']
['Damaged buildings in top left and center top.', 'Damaged buildings are concentrated in the top left and center top, indicating a possible cause or target of the damage.']
['The construction sites in the center of the image are interposed by buildings in the top right and bottom right.', 'Buildings are dispersed across the image, except for one in the middle bottom where a construction site is located.']
['Hut or tent separate in top right', 'The hut or tent in the top right stands out amidst the permanent structures, suggesting a different social or economic status.']
